# Linear regression in PyTorch

Markus Enzweiler, markus.enzweiler@hs-esslingen.de

This is a demo used in a Computer Vision & Machine Learning lecture. Feel free to use and contribute.

## Setup

Adapt `packagePath` to point to the directory containing this notebeook.

In [ ]:
# Imports
import sys
import os

In [ ]:
# Package Path
package_path = "./" # local

# Colab specific stuff below.
# If you use Colab, change package_path to a path on your Google drive

def check_for_colab():
  try:
      import google.colab
      return True
  except ImportError:
      return False

# Running on Colab?
on_colab = check_for_colab()

# Google drive mount point
gdrive_mnt = '/content/drive'

# Mount Google Drive if on Colab
if on_colab:
  from google.colab import drive
  drive.mount(gdrive_mnt, force_remount=True)
  package_path = f"{gdrive_mnt}/MyDrive/colab/lecture_examples/cv-ml-lecture-notebooks/linear_regression/torch"   # Colab


print(f"Package path: {package_path}")

In [ ]:
# Install requirements in the current Jupyter kernel
req_file = os.path.join(package_path, "requirements.txt")
if os.path.exists(req_file):
    !{sys.executable} -m pip install -r {req_file}
else:
    print(f"Requirements file not found: {req_file}")

In [ ]:
# Now we should be able to import the additional packages
import torch
import numpy as np
import matplotlib.pyplot as plt

# Set the random seed for reproducibility
torch.manual_seed(42);


## Linear regression

### Create some data based on adding noise to a known linear function

In [ ]:
# Creating a function f(x) with a slope of 2 and bias of 1, e.g. f(x) = 2x + 1
# and added Gaussian noise

# True parameters
w_true = 2
b_true = 1
params_true = torch.tensor([w_true, b_true])

X = torch.arange(-5, 5, 0.1)
Y = w_true*X + b_true + 2 * torch.randn(X.shape)

# Visualize
plt.scatter(X, Y, alpha=0.5)
plt.title("Scatter plot of f(x) = 2x + 1 + Gaussian noise")
plt.show()

### Linear model and loss

Our linear regression model is $ y = f(x) = w \cdot x + b$. We solve for $w$ and $b$ using gradient descent. 

In [ ]:
def lin_model(params, x):
    w,b = params
    return w*x + b

We uses mean squared error loss between the predictions of our model and true values. 

In [ ]:
def loss_fn(y_pred, y):   
    return torch.mean(torch.square(y - y_pred))

### Optimization via gradient descent

Initialize parameters $w$ and $b$ randomly.

In [ ]:
params = 1e-2 * torch.randn(2, dtype=torch.float32)

# we track gradients for the parameters w and b
params.requires_grad_()

w,b = params
print(f"Initial weight : {w}")
print(f"Initial bias   : {b}")

Optimize via gradient descent

In [ ]:
# Hyperparameters
num_iters = 10000
learning_rate = 3e-4

# Loop over the number of iterations
for it in range(num_iters):

    # predict y from x
    Y_pred = lin_model(params, X)

    # Compute the loss
    loss = loss_fn(Y_pred, Y)

    # Gradient of loss function w.r.t parameters
    loss.backward()
  
    # update parameters via gradient descent update rules
    with torch.no_grad():
        params -= learning_rate * params.grad
        params.grad.zero_()
  
    # Give some status output once in a while
    if it % 500 == 0 or it == num_iters - 1:
        w,b = params  
        error_norm = torch.sum(torch.square(params - params_true)) ** 0.5
        print(f"Iteration {it:5d} | Loss {loss.item():>10.5f} | " 
              f"w {w.item():> 8.5f} | b {b.item():> 8.5f} | Error norm {error_norm.item():>.5f}")
    
w,b = params    
print(f"Final weight after optimization : {w.item():.5f} (true: {w_true})")
print(f"Final bias after optimization   : {b.item():.5f} (true: {b_true})")       

Visualize linear fit

In [ ]:
# Visualize
plt.scatter(X, Y, alpha=0.5)
plt.title("Scatter plot of f(x) = 2x + 1 + Gaussian noise")

# Plot the recovered line
Y_model = lin_model(params, X)
plt.plot(X.tolist(), Y_model.tolist(), color='red')

plt.legend(["data", f"f(x) = {w.item():.3f}x + {b.item():.3f}"])
plt.show()